In [2]:
import sys
import pandas as pd
import numpy as np

sys.path

['/Users/mirandavincent/econ4161/econ4161-finalproject',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python311.zip',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload',
 '',
 '/Users/mirandavincent/Library/Python/3.11/lib/python/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages']

In [65]:
df = pd.read_csv('401k.csv')
df

,ira,a401,hval,hmort,hequity,nifa,net_nifa,tfa,net_tfa,tfa_he,...,educ,db,marr,male,twoearn,e401,p401,pira,net_n401,hown
0,0,0,69000,60150,8850,100,-3300,100,-3300,5550,...,12,0,1,0,0,0,0,0,-3300,1
1,0,0,78000,20000,58000,61010,61010,61010,61010,119010,...,16,0,0,0,0,0,0,0,61010,1
2,1800,0,200000,15900,184100,7549,7049,9349,8849,192949,...,11,0,1,1,1,0,0,1,8849,1
3,0,0,0,0,0,2487,-6013,2487,-6013,-6013,...,15,0,1,0,1,0,0,0,-6013,0
4,0,0,300000,90000,210000,10625,-2375,10625,-2375,207625,...,12,1,0,0,0,0,0,0,-2375,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9910,0,172,0,0,0,0,0,172,172,172,...,14,1,0,0,0,1,1,0,0,0
9911,0,4500,53000,49000,4000,936,-3664,5436,836,4836,...,12,1,1,1,0,1,1,0,-3664,1
9912,0,0,250000,150000,100000,10150,6150,10150,6150,106150,...,16,0,1,0,1,1,0,0,6150,1
9913,0,15000,63000,59000,4000,3499,-501,18499,14499,18499,...,18,1,1,0,1,1,1,0,-501,1


In [66]:
# Generate log income column -- for two rows income is negative so
# resulting values of NaN need to be dropped. Meanwhile for three
# rows income == 0, resulting in loginc of -inf.
df['loginc'] = np.log(df.inc)
df.drop(
  index=df[
    (df.loginc < 0) 
    | pd.isna(df.loginc)
  ].index,
  inplace=True,
)

/Users/mirandavincent/Library/Python/3.11/lib/python/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/mirandavincent/Library/Python/3.11/lib/python/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [67]:
X_list = [
  'age', 'loginc', 'fsize', 'educ', 'marr', 'twoearn', 'db', 
  'pira', 'hown', 'ira', 'male', 'hmort', 'hequity', 'hval',
]
combinations = [
  (X_list[i], X_list[j]) for i in range(0, len(X_list)) for j in range(i + 1, len(X_list))
]

for _tuple in combinations:
  col_name = f'{_tuple[0]}^2' if _tuple[0] == _tuple[1] else f'{_tuple[0]}:{_tuple[1]}'
  X_list.append(col_name)
  if not col_name in df.columns:
    df[col_name] = df[_tuple[0]] * df[_tuple[1]]

print(f'Generated {len(combinations)} new columns of interactions.')
print(f'New X_list: {X_list}.')

Generated 91 new columns of interactions.
New X_list: ['age', 'loginc', 'fsize', 'educ', 'marr', 'twoearn', 'db', 'pira', 'hown', 'ira', 'male', 'hmort', 'hequity', 'hval', 'age:loginc', 'age:fsize', 'age:educ', 'age:marr', 'age:twoearn', 'age:db', 'age:pira', 'age:hown', 'age:ira', 'age:male', 'age:hmort', 'age:hequity', 'age:hval', 'loginc:fsize', 'loginc:educ', 'loginc:marr', 'loginc:twoearn', 'loginc:db', 'loginc:pira', 'loginc:hown', 'loginc:ira', 'loginc:male', 'loginc:hmort', 'loginc:hequity', 'loginc:hval', 'fsize:educ', 'fsize:marr', 'fsize:twoearn', 'fsize:db', 'fsize:pira', 'fsize:hown', 'fsize:ira', 'fsize:male', 'fsize:hmort', 'fsize:hequity', 'fsize:hval', 'educ:marr', 'educ:twoearn', 'educ:db', 'educ:pira', 'educ:hown', 'educ:ira', 'educ:male', 'educ:hmort', 'educ:hequity', 'educ:hval', 'marr:twoearn', 'marr:db', 'marr:pira', 'marr:hown', 'marr:ira', 'marr:male', 'marr:hmort', 'marr:hequity', 'marr:hval', 'twoearn:db', 'twoearn:pira', 'twoearn:hown', 'twoearn:ira', 'twoe

In [84]:
from sklearn.preprocessing import StandardScaler

# Add p401 to list of controls such that outcome feature selection via Lasso downgrades
# importance of features correlated with p401 (which will be recovered by second p401 ~ controls)
# regression.
X = df[X_list + ['p401']]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled)
X_scaled

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,-0.972233,-0.081357,1.387857,-0.429960,0.808282,-0.784425,-0.609923,-0.565464,0.757968,-0.359040,...,-0.236869,-0.239663,-0.273750,-0.241018,-0.215447,-0.264356,-0.276966,0.008290,-0.380303,-0.595454
1,1.058676,0.126164,1.387857,0.994258,-1.237192,-0.784425,-0.609923,-0.565464,0.757968,-0.359040,...,-0.236869,-0.239663,-0.273750,-0.241018,-0.215447,-0.264356,-0.110627,-0.311182,-0.069070,-0.595454
2,0.865256,0.785187,0.087643,-0.786015,0.808282,1.274820,-0.609923,1.768459,0.757968,-0.172505,...,-0.195145,0.045233,-0.050404,0.559384,7.440632,5.039445,0.357696,-0.111385,2.499466,-0.595454
3,-1.262363,0.584678,0.737750,0.638204,0.808282,1.274820,-0.609923,-0.565464,-1.319317,-0.359040,...,-0.236869,-0.239663,-0.273750,-0.241018,-0.215447,-0.264356,-0.418038,-0.503579,-0.428869,-0.595454
4,0.091577,0.147153,0.087643,-0.429960,-1.237192,-0.784425,1.639551,-0.565464,0.757968,-0.359040,...,-0.236869,-0.239663,-0.273750,-0.241018,-0.215447,-0.264356,4.590649,2.826371,4.581590,-0.595454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9905,-0.682103,-1.111903,-1.212571,0.282149,-1.237192,-0.784425,1.639551,-0.565464,-1.319317,-0.359040,...,-0.236869,-0.239663,-0.273750,-0.241018,-0.215447,-0.264356,-0.418038,-0.503579,-0.428869,1.679391
9906,-0.778813,0.377096,0.087643,-0.429960,0.808282,-0.784425,1.639551,-0.565464,0.757968,-0.359040,...,-0.236869,-0.239663,-0.273750,2.225630,-0.049101,1.141152,-0.366096,-0.183287,-0.412008,1.679391
9907,-0.682103,1.040221,0.737750,0.994258,0.808282,1.274820,-0.609923,-0.565464,0.757968,-0.359040,...,-0.236869,-0.239663,-0.273750,-0.241018,-0.215447,-0.264356,3.557110,4.121351,1.559408,-0.595454
9908,-0.005133,0.888339,0.087643,1.706368,0.808282,1.274820,1.639551,-0.565464,0.757968,-0.359040,...,-0.236869,-0.239663,-0.273750,-0.241018,-0.215447,-0.264356,-0.355496,-0.045156,-0.408827,1.679391


In [75]:
from sklearn.linear_model import LassoCV

def run_lasso(X: pd.DataFrame, y: pd.Series) -> tuple[LassoCV, np.ndarray]:
  reg = LassoCV(
    cv=10, 
    random_state=0, 
    n_jobs=-1, 
    # Lift default tolerance to try to converge faster (this LassoCV uses coordinate descent to find best alpha)
    tol=1e-2 
  ).fit(X, y)
  return reg, np.where(np.abs(reg.coef_) > 1e-6)[0]

In [100]:
reg, coef_indices = run_lasso(X_scaled, df.tw)
selected_covariates = X.columns[coef_indices]
print(f'Found {selected_covariates} ({len(selected_covariates)}).')

Found Index(['age', 'loginc', 'fsize', 'educ', 'marr', 'db', 'pira', 'hown', 'ira',
       'hmort', 'hequity', 'hval', 'age:loginc', 'age:fsize', 'age:educ',
       'age:twoearn', 'age:db', 'age:pira', 'age:ira', 'age:hmort', 'age:hval',
       'loginc:marr', 'loginc:pira', 'loginc:ira', 'loginc:male',
       'loginc:hmort', 'loginc:hequity', 'loginc:hval', 'fsize:marr',
       'fsize:twoearn', 'fsize:db', 'fsize:pira', 'fsize:ira', 'fsize:male',
       'fsize:hmort', 'fsize:hval', 'educ:marr', 'educ:twoearn', 'educ:db',
       'educ:pira', 'educ:hown', 'educ:ira', 'educ:male', 'educ:hequity',
       'marr:db', 'marr:pira', 'marr:hown', 'marr:male', 'marr:hequity',
       'marr:hval', 'twoearn:db', 'twoearn:hown', 'twoearn:ira',
       'twoearn:male', 'twoearn:hequity', 'twoearn:hval', 'db:pira', 'db:hown',
       'db:ira', 'db:male', 'db:hmort', 'pira:hown', 'pira:ira', 'pira:hmort',
       'pira:hval', 'hown:ira', 'hown:male', 'hown:hmort', 'hown:hequity',
       'hown:hval', 'ira:ma

/Users/mirandavincent/Library/Python/3.11/lib/python/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.064e+12, tolerance: 1.233e+12
  model = cd_fast.enet_coordinate_descent(


In [127]:
from sklearn.linear_model import LogisticRegressionCV

def run_logistic_reg(X: pd.DataFrame, y: pd.Series) -> tuple[LassoCV, np.ndarray]:
  reg = LogisticRegressionCV(
    penalty='l1',
    cv=10,
    random_state=0,
    solver='liblinear',
    tol=1e-2
  ).fit(X, y)
  return reg, np.where(np.abs(reg.coef_[0]) > 1e-6)[0]

In [128]:
# Simply construct the dataframe from the original set of controls (no p401).
X2_scaled = X_scaled.iloc[:, 1:-1]
log_reg, coef_indices_2 = run_logistic_reg(X2_scaled, df.p401)

In [129]:
selected_covariates_log = X.columns[coef_indices_2]
selected_covariates_log

Index(['age', 'loginc', 'fsize', 'educ', 'marr', 'twoearn', 'db', 'pira',
       'hown', 'male', 'hmort', 'age:loginc', 'age:fsize', 'age:educ',
       'age:marr', 'age:twoearn', 'age:db', 'age:pira', 'age:hown', 'age:ira',
       'age:male', 'age:hval', 'loginc:fsize', 'loginc:educ', 'loginc:twoearn',
       'loginc:pira', 'loginc:hown', 'loginc:ira', 'loginc:male',
       'loginc:hequity', 'loginc:hval', 'fsize:educ', 'fsize:twoearn',
       'fsize:db', 'fsize:pira', 'fsize:hown', 'fsize:ira', 'fsize:male',
       'fsize:hequity', 'fsize:hval', 'educ:marr', 'educ:twoearn', 'educ:hown',
       'educ:ira', 'educ:male', 'educ:hmort', 'educ:hequity', 'marr:twoearn',
       'marr:db', 'marr:pira', 'marr:hown', 'marr:ira', 'marr:male',
       'marr:hmort', 'marr:hval', 'twoearn:db', 'twoearn:hown', 'twoearn:ira',
       'twoearn:male', 'twoearn:hmort', 'twoearn:hequity', 'twoearn:hval',
       'db:pira', 'db:hown', 'db:ira', 'db:male', 'db:hequity', 'db:hval',
       'pira:hown', 'pira:ira

In [137]:
union_cols = list(set(selected_covariates) | set(selected_covariates_log))
X_union = X[union_cols]

In [162]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(
  X_union, 
  df.tw, 
  test_size=0.1, 
  random_state=0
)

X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

res = sm.OLS(y_train, X_train).fit()

y_pred = res.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root mean squared error: {rmse}')
print(f'Coefficient on P401: {res.params["p401"]}.')

Root mean squared error: 61830.7161319639
Coefficient on P401: 11394.39924963918.


In [163]:
res.params

const           357848.811296
fsize:educ         363.350287
male:hequity         0.117392
twoearn          87048.492402
male:hmort          -0.019197
                    ...      
hval                -0.340195
male              4205.570875
marr:male        -2863.948924
ira:hmort            0.448598
fsize:marr        -985.892649
Length: 102, dtype: float64